In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset

from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler()


In [2]:
df = pd.read_pickle('../data/PandasData/Modified/PID001_NSL.pkl')
numeric_df = df.select_dtypes(include=[np.number])
numeric_df.drop(columns=['Timestamp', 'start_station_X', 'start_station_Y', 'end_station_X', 'end_station_Y',
       'distance_from_start_station_X', 'distance_from_start_station_Y',
       'distance_from_end_station_X', 'distance_from_end_station_Y','AGV distance X', 'AGV distance Y', ], inplace=True)

In [34]:
timeseries = numeric_df.values.astype('float32')

# train-test split for time series
train_size = int(len(timeseries) * 0.67)
test_size = len(timeseries) - train_size
train, test = timeseries[:train_size], timeseries[train_size:]

scaler = StandardScaler()
scaled_train = scaler.fit_transform(train)
scaled_test = scaler.transform(test)

def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset
    
    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset)-lookback):
        feature = dataset[i:i+lookback]
        target = dataset[i+1:i+lookback+1]
        X.append(feature)
        y.append(target)
    return torch.tensor(X), torch.tensor(y)
 
# lookback = 4
# X_train, y_train = create_dataset(train, lookback=lookback)
# X_test, y_test = create_dataset(test, lookback=lookback)

lookback = 4
X_train, y_train = create_dataset(scaled_train, lookback=lookback)
X_test, y_test = create_dataset(scaled_test, lookback=lookback)

In [48]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([54, 4, 21]),
 torch.Size([54, 4, 21]),
 torch.Size([25, 4, 21]),
 torch.Size([25, 4, 21]))

In [20]:
class TraPredModel(nn.Module):
    def __init__(self, input_size = None, lookback = None):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=50, num_layers=lookback, batch_first=True)
        self.linear = nn.Linear(50, 2)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

In [96]:
model = TraPredModel(input_size=numeric_df.shape[1], lookback=lookback)
for name, param in model.named_parameters():
    if 'weight_ih' in name:  # Input-hidden weights of the LSTM
        torch.nn.init.xavier_uniform_(param.data)
    elif 'weight_hh' in name:  # Hidden-hidden weights of the LSTM
        torch.nn.init.orthogonal_(param.data)
    elif 'weight' in name:  # Weights of linear layers
        torch.nn.init.kaiming_uniform_(param.data)
    elif 'bias' in name:  # Biases
        param.data.fill_(0.01)



optimizer = optim.AdamW(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()
train_ds = DataLoader(TensorDataset(X_train, y_train), shuffle=True, batch_size=8)
test_ds = DataLoader(TensorDataset(X_test, y_test), shuffle=False, batch_size=8)


In [102]:
n_epochs = 100
model = TraPredModel(input_size=numeric_df.shape[1], lookback=lookback)

for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in train_ds:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch[:, :, -2:])
        optimizer.zero_grad()
        print(loss)
        loss.backward()
        optimizer.step()

    # Validation
    if epoch % 2 == 0:
        with torch.no_grad():
            all_test = len(test_ds)
            test_rmse_all = []
            for X_test_batch, y_test_batch in test_ds:
                y_pred = model(X_test_batch)

                test_rmse = np.sqrt(loss_fn(y_pred, y_test_batch[:, :, -2:]))
                if not torch.isnan(test_rmse):
                    test_rmse_all.append(test_rmse.item())

            print(test_rmse_all)
            
            print("Epoch %d: test RMSE %.4f" % (epoch, sum(test_rmse_all)/all_test))
    # break
 

tensor(nan, grad_fn=<MseLossBackward0>)
tensor(1.4835, grad_fn=<MseLossBackward0>)
tensor(0.6602, grad_fn=<MseLossBackward0>)
tensor(1.0034, grad_fn=<MseLossBackward0>)
tensor(1.0708, grad_fn=<MseLossBackward0>)
tensor(0.6273, grad_fn=<MseLossBackward0>)
tensor(1.2517, grad_fn=<MseLossBackward0>)
[0.8744215369224548, 3.962294578552246, 6.719480514526367]
Epoch 0: test RMSE 2.8890
tensor(1.1619, grad_fn=<MseLossBackward0>)
tensor(0.5570, grad_fn=<MseLossBackward0>)
tensor(1.2624, grad_fn=<MseLossBackward0>)
tensor(nan, grad_fn=<MseLossBackward0>)
tensor(1.0986, grad_fn=<MseLossBackward0>)
tensor(1.3755, grad_fn=<MseLossBackward0>)
tensor(0.8585, grad_fn=<MseLossBackward0>)
tensor(0.9863, grad_fn=<MseLossBackward0>)
tensor(0.7495, grad_fn=<MseLossBackward0>)
tensor(0.6802, grad_fn=<MseLossBackward0>)
tensor(nan, grad_fn=<MseLossBackward0>)
tensor(1.3442, grad_fn=<MseLossBackward0>)
tensor(0.6503, grad_fn=<MseLossBackward0>)
tensor(1.1569, grad_fn=<MseLossBackward0>)
[0.8744215369224548, 

In [77]:
test_rmse.item() == float('nan')

False

In [28]:
y_batch[:, :, -2:]

tensor([[[2.8357e-04, 6.6137e-03],
         [2.6307e-04, 6.1712e-03],
         [2.1249e-04, 5.0802e-03],
         [1.6734e-04, 4.1370e-03]],

        [[3.2650e-01, 3.0699e-01],
         [2.7231e-01, 2.7397e-01],
         [2.2124e-01, 1.9812e-01],
         [1.6293e-01, 1.8219e-01]],

        [[1.6734e-04, 4.1370e-03],
         [1.6734e-04, 4.1370e-03],
         [1.6734e-04, 4.1370e-03],
         [1.6734e-04, 4.1370e-03]],

        [[1.0000e+00, 7.7372e-02],
         [9.9989e-01, 9.5654e-02],
         [9.9989e-01, 9.5654e-02],
         [9.9989e-01, 9.5654e-02]],

        [[1.3000e-04, 3.1300e-03],
         [1.3000e-04, 3.1300e-03],
         [1.3000e-04, 3.1300e-03],
         [1.2544e-04, 3.0155e-03]],

        [[1.6734e-04, 4.1370e-03],
         [1.3232e-04, 3.1929e-03],
         [1.3000e-04, 3.1300e-03],
         [1.3000e-04, 3.1300e-03]],

        [[1.6293e-01, 1.8219e-01],
         [1.1653e-01, 2.5023e-01],
         [6.7933e-02, 2.6236e-01],
         [2.3676e-02, 1.6142e-01]],

      

In [29]:
y_pred

tensor([[[-0.0786,  0.0392],
         [-0.0636,  0.0486],
         [-0.0565,  0.0533],
         [-0.0530,  0.0557]],

        [[-0.0786,  0.0392],
         [-0.0636,  0.0486],
         [-0.0564,  0.0533],
         [-0.0530,  0.0556]],

        [[-0.0786,  0.0392],
         [-0.0636,  0.0486],
         [-0.0565,  0.0534],
         [-0.0530,  0.0557]],

        [[-0.0785,  0.0392],
         [-0.0634,  0.0485],
         [-0.0560,  0.0532],
         [-0.0524,  0.0555]],

        [[-0.0786,  0.0392],
         [-0.0636,  0.0486],
         [-0.0565,  0.0534],
         [-0.0530,  0.0557]],

        [[-0.0786,  0.0392],
         [-0.0636,  0.0486],
         [-0.0565,  0.0534],
         [-0.0530,  0.0557]],

        [[-0.0786,  0.0392],
         [-0.0635,  0.0486],
         [-0.0564,  0.0534],
         [-0.0530,  0.0557]],

        [[-0.0786,  0.0392],
         [-0.0636,  0.0487],
         [-0.0565,  0.0534],
         [-0.0530,  0.0557]]], grad_fn=<AddBackward0>)

In [7]:
loss

tensor(27025266., grad_fn=<MseLossBackward0>)

In [103]:
y_batch[:, :, -2:]

tensor([[[2036.3741, 6182.6597],
         [1838.5870, 6191.6587],
         [1658.4749, 6116.8018],
         [1571.1934, 6015.2192]],

        [[1563.2740, 6001.9951],
         [1563.2740, 6001.9951],
         [1563.1907, 6001.6665],
         [1562.9847, 6000.8579]],

        [[1563.1907, 6001.6665],
         [1562.9847, 6000.8579],
         [1562.8010, 6000.1582],
         [1562.8010, 6000.1582]],

        [[1562.8010, 6000.1582],
         [1562.8010, 6000.1582],
         [1562.8010, 6000.1582],
         [1562.8010, 6000.1582]],

        [[4166.2915, 6263.7432],
         [3882.6104, 6264.6323],
         [3626.2195, 6230.0762],
         [3382.0386, 6161.8926]],

        [[2462.5181, 6144.0171],
         [2225.2104, 6132.2021],
         [2036.3741, 6182.6597],
         [1838.5870, 6191.6587]],

        [[1562.6490, 5999.4111],
         [1562.6490, 5999.4111],
         [1562.6490, 5999.4111],
         [1562.6490, 5999.4111]],

        [[5629.0020, 6020.4014],
         [5629.1602, 6019.796

In [107]:
loss_fn(y_pred, y_batch[:, :, -2:])

tensor(29356700., grad_fn=<MseLossBackward0>)

In [108]:
y_batch_last2

tensor([[[1562.6305, 5999.3267],
         [1562.5750, 5999.0732],
         [1562.3723, 5998.1821],
         [1562.3480, 5998.0752]],

        [[1563.2740, 6001.9951],
         [1563.2740, 6001.9951],
         [1563.2740, 6001.9951],
         [1563.2740, 6001.9951]],

        [[3626.2195, 6230.0762],
         [3382.0386, 6161.8926],
         [3129.8562, 6181.4565],
         [2890.8782, 6224.7598]],

        [[5631.3882, 6068.0288],
         [5631.3882, 6068.0288],
         [5631.3882, 6068.0288],
         [5630.5522, 6069.1782]],

        [[3382.0386, 6161.8926],
         [3129.8562, 6181.4565],
         [2890.8782, 6224.7598],
         [2670.3411, 6200.2725]],

        [[1562.8010, 6000.1582],
         [1562.8010, 6000.1582],
         [1562.8010, 6000.1582],
         [1562.6586, 5999.4580]],

        [[5629.0020, 6020.4014],
         [5629.1602, 6019.7969],
         [5631.8296, 6054.4697],
         [5631.3882, 6068.0288]],

        [[2890.8782, 6224.7598],
         [2670.3411, 6200.272

In [31]:
loss_fn(y_pred_last2, y_batch_last2)

/Users/shawn/opt/anaconda3/envs/pytorch/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([8, 2, 21])) that is different to the input size (torch.Size([8, 2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(nan, grad_fn=<MseLossBackward0>)

In [34]:
len(numeric_df.columns)

21

In [44]:
y_pred

tensor([[[nan],
         [nan],
         [nan],
         [nan]],

        [[nan],
         [nan],
         [nan],
         [nan]],

        [[nan],
         [nan],
         [nan],
         [nan]],

        [[nan],
         [nan],
         [nan],
         [nan]],

        [[nan],
         [nan],
         [nan],
         [nan]],

        [[nan],
         [nan],
         [nan],
         [nan]],

        [[nan],
         [nan],
         [nan],
         [nan]],

        [[nan],
         [nan],
         [nan],
         [nan]]], grad_fn=<AddBackward0>)

In [45]:
y_batch[:, :, -2:]

tensor([[[5559.4497, 6606.9014],
         [5542.7568, 6721.1694],
         [5536.9160, 6738.7012],
         [5471.5371, 6733.4746]],

        [[2890.8782, 6224.7598],
         [2670.3411, 6200.2725],
         [2462.5181, 6144.0171],
         [2225.2104, 6132.2021]],

        [[1562.8010, 6000.1582],
         [1562.8010, 6000.1582],
         [1562.8010, 6000.1582],
         [1562.8010, 6000.1582]],

        [[5077.3340, 6578.1284],
         [4839.5220, 6500.1108],
         [4633.7700, 6408.1240],
         [4426.0840, 6316.3164]],

        [[3882.6104, 6264.6323],
         [3626.2195, 6230.0762],
         [3382.0386, 6161.8926],
         [3129.8562, 6181.4565]],

        [[1562.6490, 5999.4111],
         [1562.6490, 5999.4111],
         [1562.6490, 5999.4111],
         [1562.6305, 5999.3267]],

        [[2036.3741, 6182.6597],
         [1838.5870, 6191.6587],
         [1658.4749, 6116.8018],
         [1571.1934, 6015.2192]],

        [[1562.6305, 5999.3267],
         [1562.5750, 5999.073

In [36]:
y_batch[:, :, -2:].shape

torch.Size([8, 4, 2])

In [12]:
y_batch_last2

tensor([[[ 2.4965e-05,  6.0469e-04,  6.0521e-04,  2.0769e+00,  9.1807e-01,
           2.2707e+00, -2.0769e+00, -9.1807e-01,  0.0000e+00,  2.0000e+00,
           4.0000e+00,  1.4727e+03,  1.3739e+03,  5.3032e+02, -9.0132e-01,
          -2.1056e-02, -4.2835e-01,  8.4683e+02,  6.6767e+03,  4.4261e+03,
           6.3163e+03],
         [ 2.3611e-05,  6.1139e-04,  6.1184e-04,  2.5979e+00,  5.2573e-01,
           2.6506e+00, -2.5979e+00, -5.2573e-01,  0.0000e+00,  2.0000e+00,
           4.0000e+00,  1.7021e+03,  1.6337e+03,  4.7774e+02, -8.6969e-01,
           6.4958e-02, -4.8786e-01,  8.4683e+02,  6.6767e+03,  4.1663e+03,
           6.2637e+03]],

        [[ 2.9754e-05,  6.0510e-04,  6.0583e-04,  2.4366e-04,  1.0711e-03,
           1.0985e-03, -2.4366e-04, -1.0711e-03,  0.0000e+00,  2.0000e+00,
           2.0000e+00,  2.0262e+02,  4.2652e+01,  1.9807e+02, -6.0803e-01,
          -7.7583e-01, -1.2950e-01,  8.4674e+02,  6.6746e+03,  1.5623e+03,
           5.9981e+03],
         [ 2.9167e-05,  6.